In [78]:
from kiteconnect import KiteConnect
from selenium import webdriver
import time
import os
import re

In [79]:
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
import os
import sqlite3

In [80]:
apikey=""
apisecret=""

In [81]:
kite=KiteConnect(api_key=apikey)

In [82]:
print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=kdwok1gk1epq3k3k&v=3


In [83]:
data=kite.generate_session("0sD4stclQrUmssLaZY78ZXjpYwmXdURR",api_secret=apisecret)

In [84]:
data

{'user_type': 'individual',
 'email': 'the.ravina@gmail.com',
 'user_name': 'Ravina Vinayak More',
 'user_shortname': 'Ravina',
 'broker': 'ZERODHA',
 'exchanges': ['CDS', 'BSE', 'NSE', 'NFO', 'BFO', 'MF'],
 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'],
 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'],
 'avatar_url': None,
 'user_id': 'LF9990',
 'api_key': 'kdwok1gk1epq3k3k',
 'access_token': 'XJT284FfjkS6Ap303MOTtahehBObLbCt',
 'public_token': 'ZUTY0nmOHZS5Mobx2OSdM0RO0Q2YoR15',
 'refresh_token': '',
 'enctoken': 'BeLe/vbuBMyvlqyF9WyB39zeoVb4TrgoQmp74pURMEsdbQgqydGpDKswtD09zXLTHXtvif6b+r1wmzRYq+2U8/sp1QoaJmVJ1/AdDhUjkF0Xz10J9dOTLowpoEhW6rg=',
 'login_time': datetime.datetime(2023, 5, 15, 9, 9, 55),
 'meta': {'demat_consent': 'physical'}}

In [85]:
kite.set_access_token(data["access_token"])

In [86]:
kite.instruments("NSE")

[{'instrument_token': 264713,
  'exchange_token': '1034',
  'tradingsymbol': 'HANGSENG BEES-NAV',
  'name': 'HANGSENG BEES-NAV',
  'last_price': 0.0,
  'expiry': '',
  'strike': 0.0,
  'tick_size': 0.0,
  'lot_size': 0,
  'instrument_type': 'EQ',
  'segment': 'INDICES',
  'exchange': 'NSE'},
 {'instrument_token': 264969,
  'exchange_token': '1035',
  'tradingsymbol': 'INDIA VIX',
  'name': 'INDIA VIX',
  'last_price': 0.0,
  'expiry': '',
  'strike': 0.0,
  'tick_size': 0.0,
  'lot_size': 0,
  'instrument_type': 'EQ',
  'segment': 'INDICES',
  'exchange': 'NSE'},
 {'instrument_token': 260617,
  'exchange_token': '1018',
  'tradingsymbol': 'NIFTY 100',
  'name': 'NIFTY 100',
  'last_price': 0.0,
  'expiry': '',
  'strike': 0.0,
  'tick_size': 0.0,
  'lot_size': 0,
  'instrument_type': 'EQ',
  'segment': 'INDICES',
  'exchange': 'NSE'},
 {'instrument_token': 264457,
  'exchange_token': '1033',
  'tradingsymbol': 'NIFTY 200',
  'name': 'NIFTY 200',
  'last_price': 0.0,
  'expiry': '',
  '

In [87]:
inst_df=pd.DataFrame(kite.instruments("NSE"))

In [88]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [89]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(inst_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [90]:
def fetchOHLCExtended(ticker,inception_date, interval):
    """extracts historical data and outputs in the form of dataframe
       inception date string format - dd-mm-yyyy"""
    instrument = instrumentLookup(inst_df,ticker)
    from_date = dt.datetime.strptime(inception_date, '%d-%m-%Y')
    to_date = dt.date.today()
    data = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    temp = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'volume'])

    while True:
        if from_date.date() >= (dt.date.today() - dt.timedelta(100)):
            temp=pd.DataFrame(kite.historical_data(instrument,from_date,dt.date.today(),interval))
            data=pd.concat([data,temp])
#             data = data.append(pd.DataFrame(kite.historical_data(instrument,from_date,dt.date.today(),interval)),ignore_index=True)
            #time.sleep(0.2)
            break
        else:
            to_date = from_date + dt.timedelta(100)
            temp=pd.DataFrame(kite.historical_data(instrument,from_date,to_date,interval))
            data=pd.concat([data,temp])
#             data = data.append(pd.DataFrame(kite.historical_data(instrument,from_date,to_date,interval)),ignore_index=True)
            from_date = to_date
            #time.sleep(0.2)
    data.set_index("date",inplace=True)
    return data

In [ ]:
def slope(ohlc_df,n):
    df = ohlc_df.iloc[-1*n:,:]
    y = ((df["open"] + df["close"])/2).values
    x = np.array(range(n))
    y_scaled = (y - y.min())/(y.max() - y.min())
    x_scaled = (x - x.min())/(x.max() - x.min())
    x_scaled = sm.add_constant(x_scaled)
    model = sm.OLS(y_scaled,x_scaled)
    results = model.fit()
    slope = np.rad2deg(np.arctan(results.params[-1]))
    return slope

In [ ]:
def trend(ohlc_df,n):
    df = ohlc_df.copy()
    df["up"] = np.where(df["low"]>=df["low"].shift(1),1,0)
    df["dn"] = np.where(df["high"]<=df["high"].shift(1),1,0)
    if df["close"][-1] > df["open"][-1]:
        if df["up"][-1*n:].sum() >= 0.7*n:
            return "uptrend"
    elif df["open"][-1] > df["close"][-1]:
        if df["dn"][-1*n:].sum() >= 0.7*n:
            return "downtrend"
    else:
        return None


In [ ]:
ohlc = fetchOHLC("YESBANK","5minute",30)
slope(ohlc,7)
trend(ohlc,7)